In [1]:
import tensorflow as tf
from keras.src.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras import layers, models

In [3]:
#import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
tf.sysconfig.get_build_info()


OrderedDict([('is_cuda_build', False),
             ('is_rocm_build', False),
             ('is_tensorrt_build', False),
             ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll')])

In [8]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [28]:
def create_cnn(input_shape, num_classes1, num_classes2):
    input_layer = layers.Input(shape=input_shape)

    conv1 = layers.Conv2D(96, (11, 11), activation='relu', strides=4, padding='same')(input_layer)
    #lrn1 = layers.Lambda(lambda x: tf.nn.local_response_normalization(x, alpha=1e-4, beta=0.75, bias=2, depth_radius=5))(conv1)
    batch1 = tf.keras.layers.BatchNormalization()(conv1)

    pool1 = layers.MaxPooling2D((3, 3), strides=2)(batch1)

    conv2 = layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(pool1)
    #lrn2 = layers.Lambda(lambda x: tf.nn.local_response_normalization(x, alpha=1e-4, beta=0.75, bias=2, depth_radius=5))(conv2)
    batch3 = tf.keras.layers.BatchNormalization()(conv2)

    pool2 = layers.MaxPooling2D((3, 3), strides=2)(batch3)

    conv3 = layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(pool2)
    batch5 = tf.keras.layers.BatchNormalization()(conv3)

    conv4 = layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(batch5)
    batch6 = tf.keras.layers.BatchNormalization()(conv4)

    conv5 = layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(batch6)
    batch7 = tf.keras.layers.BatchNormalization()(conv5)

    pool3 = layers.MaxPooling2D((3, 3), strides=2, padding='same')(batch7)

    # Flatten layer
    flatten = layers.Flatten()(pool3)

    # Fully connected layers
    dense1 = layers.Dense(2048, activation='relu')(flatten)
    batch9 = tf.keras.layers.BatchNormalization()(dense1)
    dense2 = layers.Dense(2048, activation='relu')(batch9)
    batch10 = tf.keras.layers.BatchNormalization()(dense2)

    # Branch 1
    output1 = layers.Dense(num_classes1, activation='softmax', name='output1')(batch10)

    # Branch 2
    output2 = layers.Dense(num_classes2, activation='sigmoid', name='output2')(batch10)

    model = models.Model(inputs=input_layer, outputs=[output1, output2])

    return model

In [29]:

input_shape = (28, 28, 1)
num_classes1 = 10
num_classes2 = 5
model = create_cnn(input_shape, num_classes1, num_classes2)

# Compile the model
model.compile(optimizer='adam',
              loss={'output1': 'sparse_categorical_crossentropy', 'output2': 'sparse_categorical_crossentropy'},
              metrics={'output1': 'accuracy', 'output2': 'accuracy'})

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 7, 7, 96)             11712     ['input_2[0][0]']             
                                                                                                  
 batch_normalization_40 (Ba  (None, 7, 7, 96)             384       ['conv2d_1[0][0]']            
 tchNormalization)                                                                                
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 3, 3, 96)             0         ['batch_normalization_40[